In [1]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from dotenv import load_dotenv, find_dotenv

from langchain.tools import E2BDataAnalysisTool
import os
from langchain.agents import AgentExecutor
from langchain.agents import tool, initialize_agent, AgentType, Tool
from langchain.chat_models import AzureChatOpenAI

In [2]:
load_dotenv(find_dotenv())

True

In [29]:
E2B_API_KEY = os.getenv("E2B_API_KEY")

In [30]:
# from langchain_community.tools.tavily_search import TavilySearchResults
# tavily_tool = TavilySearchResults(
#     description="A search engine. Useful for when you don't know something and want to find it out.",)

In [31]:
e2b_data_analysis_tool = E2BDataAnalysisTool(api_key=E2B_API_KEY)
#e2b_data_analysis_tool.install_python_packages("geopy")

Exception in thread e2b-sandbox-refresh:
Traceback (most recent call last):
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\e2b\sandbox\sandbox_connection.py", line 433, in _refresh
    api.sandboxes_sandbox_id_refreshes_post(
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\pydantic\validate_call_decorator.py", line 58, in wrapper_function
    return validate_call_wrapper(*args, **kwargs)
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\pydantic\_internal\_validate_call.py", line 81, in __call__
    res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\e2b\api\v2\client\api\sandboxes_api.py", line 838, in sandboxes_sandbox_id_refreshes_post
    return self.api_client.response_deserialize(
  File "c:\Users\justa\OneDrive

In [32]:
@tool
def install_python_package_to_e2b(package_name: str):
    """
    Installs a python package to the E2B environment. This is useful for when you need to install a package that is not already installed in the E2B environment.
    """
    e2b_data_analysis_tool.install_python_packages(package_name)

In [33]:
tools = [e2b_data_analysis_tool, install_python_package_to_e2b]

Can a normal gpt (not assitant do the same?)

In [7]:
txt = """
'Aircraft ids: [\'KL204\', \'KL420\']\nInfo on KL204 B744 index = 0\nPos: N52\'00\'0.0"  E004\'00\'0.0"\nHdg: 270   Trk: 270\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 350/498/498 kts   M: 0.827\n\nInfo on KL420 B744 index = 1\nPos: N52\'00\'0.0"  E003\'12\'0.0"\nHdg: 090   Trk: 090\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 350/498/498 kts   M: 0.827\n\n'
"""

question = f"Based on this info what wil be the position of the closest approach point between these aircraft and after how much time will it happen: \n\n {txt}"

In [8]:
question

'Based on this info what wil be the position of the closest approach point between these aircraft and after how much time will it happen: \n\n \n\'Aircraft ids: [\'KL204\', \'KL420\']\nInfo on KL204 B744 index = 0\nPos: N52\'00\'0.0"  E004\'00\'0.0"\nHdg: 270   Trk: 270\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 350/498/498 kts   M: 0.827\n\nInfo on KL420 B744 index = 1\nPos: N52\'00\'0.0"  E003\'12\'0.0"\nHdg: 090   Trk: 090\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 350/498/498 kts   M: 0.827\n\n\'\n'

In [11]:
# #llm = ChatOpenAI(model='gpt-4-0125-preview', temperature=0)

In [12]:
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
llm = ChatAnthropic(model='claude-3-opus-20240229', anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"))
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))

In [13]:


# llm = AzureChatOpenAI(
#     openai_api_base='https://air-traffic-management-llm.openai.azure.com/openai',
#     openai_api_version="2023-03-15-preview",
#     deployment_name="thesis",
#     openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
#     openai_api_type="azure",
#     model_name="gpt-4"
# )
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate


prompt_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""
prompt = PromptTemplate.from_template(prompt_template)


tools = [
    Tool(
        name="install python package to e2b",
        func=lambda x: install_python_package_to_e2b(x),
        description="useful when you need to install a python package to the E2B environment. When importing libraries in e2b first install the python package using this tool."
    ),
    e2b_data_analysis_tool.as_tool()
]


react = create_react_agent(
    tools=tools, llm=llm,  prompt=prompt)

agent_executor = AgentExecutor(
    agent=react, tools=tools, verbose=True)
agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...
To solve this problem, we need to calculate the point of closest approach between the two aircraft based on their positions, headings, and speeds. Given that both aircraft are flying at the same altitude and have a constant ground speed, we can simplify the problem to a 2D plane. The first aircraft (KL204) is flying west (270 degrees) and the second aircraft (KL420) is flying east (90 degrees). Their speeds are the same, so the relative speed between them is the sum of their speeds. We can calculate the time until the closest approach by dividing the initial distance between them by their relative speed. The position of the closest approach can then be determined based on the speed and heading of each aircraft.

Action: e2b_data_analysis

Action Input:
```python
from geopy.distance import geodesic
import numpy as np

# Positions of the aircrafts
pos_KL204 = (52.0, 4.0)  # N52'00'0.0" E004'00'0.0"
pos_KL420 = (52.0, 3.2)  # N52'00'0.0" E003'12'0.0

SyntaxError: invalid syntax (<unknown>, line 1)

In [40]:
from langchain_openai import ChatOpenAI
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_anthropic.experimental import ChatAnthropicTools



# gpt-4-0125-preview
# Create a chat prompt template
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
llm = ChatAnthropicTools(model='claude-3-opus-20240229',
                         anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"))

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful air taffic control operator, that can also use tools to answer questions and perform tasks",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [41]:
agent_executor.invoke(
    {"input": question})



> Entering new AgentExecutor chain...


Error in StdOutCallbackHandler.on_agent_action callback: AttributeError("'tuple' object has no attribute 'log'")


AttributeError: 'tuple' object has no attribute 'tool'

OLLAMA

In [10]:
from langchain_community.llms import Ollama
llm = Ollama(base_url="https://ollama.junzis.com", model="gemma")

'The largest airport in the world is Hartsfield-Jackson Atlanta International Airport (ATL) in Atlanta, Georgia, USA.'

In [16]:
llm.invoke(question)

'Based on this information, the closest approach point between the two aircraft will be at the same position as their current position, which is:\n\n**N52\'00\'0.0" E004\'00\'0.0"**\n\nThis is because the two aircraft are directly opposite each other on the same latitude line, and their altitudes are the same.\n\nThe time at which this closest approach point will happen is not included in the text, therefore I cannot provide an answer for that question.'

In [21]:
e2b_data_analysis_tool.as_tool()


Tool(name='e2b_data_analysis', description='Evaluates python code in a sandbox environment. The environment is long running and exists across multiple executions. You must send the whole script every time and print your outputs. Script should be pure python code that can be evaluated. It should be in python format NOT markdown. The code should NOT be wrapped in backticks. All python packages including requests, matplotlib, scipy, numpy, pandas, etc are available. Create and display chart using `plt.show()`.', args_schema=<class 'langchain_community.tools.e2b_data_analysis.tool.E2BDataAnalysisToolArguments'>, func=<bound method E2BDataAnalysisTool._run of E2BDataAnalysisTool(description='Evaluates python code in a sandbox environment. The environment is long running and exists across multiple executions. You must send the whole script every time and print your outputs. Script should be pure python code that can be evaluated. It should be in python format NOT markdown. The code should NO

In [43]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

e2b_data_analysis_tool = E2BDataAnalysisTool(api_key=E2B_API_KEY)
llm = ChatAnthropic(model='claude-3-opus-20240229', anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"))
#llm_gpt = ChatOpenAI(model="gpt-4-0125-preview", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
tools = [e2b_data_analysis_tool, install_python_package_to_e2b]
agent = initialize_agent(
    tools=tools,
    llm=llm_with_tools,
    verbose=True,
    handle_parsing_errors=True)
agent.run(question)



> Entering new AgentExecutor chain...

Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Here is the step-by-step solution to find the closest approach point and time between the two aircraft:

Thought: To find the closest approach point and time, I will use the aircraft positions, headings, and speeds to calculate when their paths will intersect. I can do this by setting up equations for each aircraft's position over time and solving for when the x and y positions are equal.

Action: e2b_data_analysis

Action Input:
import math
import numpy as np

# Aircraft 1 - KL204
lat1 = 52.0
lon1 = 4.0 
hdg1 = math.radians(270)
spd1 = 498 * 0.514444 # convert knots to m/s

# Aircraft 2 - KL420 
lat2 = 52.0
lon2 = 3.2
hdg2 = math.radians(90) 
spd2 = 498 * 0.514444

# Convert positions to Cartesian coordinates with (0,0) at KL204 start
x1 = 0
y1 = 0
x2 = (lon1 - lon2) * 60 * 1852 # convert longitude diff to meters
y2 = 0

# Solve for intersection point
# x1 + spd1*cos(hdg1)*t

'Based on the given information, the two aircraft KL204 and KL420 will reach their closest approach point approximately 393.5 seconds (6.5 minutes) after the stated positions.\n\nThe closest approach will occur at the intersection point of their paths, which is located at:\nLatitude: 52°00\'00.0" N\nLongitude: 004°29\'24.0" E \n\nThis is 54,440 meters (29.4 nautical miles) east of KL204\'s starting position.\n\nTo summarize:\n- Time to closest approach: 393.5 seconds (6.5 minutes)\n- Location of closest approach: 52°00\'00.0"N, 004°29\'24.0"E'

WebSocket received error while receiving messages: no close frame received or sent
Exception in thread e2b-sandbox-refresh:
Traceback (most recent call last):
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connectionpool.py", line 793, in urlopen
    response = self._make_request(
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
  File "C:\Python39\lib\http\client.py", line 1349, in getresponse
    response.begin()
  File "C:\Python39\lib\http\client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "C:\Python39\lib\http\client.py", line 277, in _read_status
    line = str(self.fp.r

In [19]:
question = question + \
    "\n\n when writing python code only write the code, no need to add extra stuff like ```python"

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI, ChatOpenAI
from langchain_anthropic import Anthropic, ChatAnthropic
# Choose the LLM to use
llm = ChatAnthropic(model='claude-3-opus-20240229',
                    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"))
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)
prompt = hub.pull("hwchase17/react")

e2b_data_analysis_tool = E2BDataAnalysisTool(api_key=E2B_API_KEY)
#e2b_data_analysis_tool.install_python_packages('math')
tools = [e2b_data_analysis_tool, install_python_package_to_e2b]

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools


def _handle_error(error) -> str:
    return str(error)[:50]

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=_handle_error
)


WebSocket received error while receiving messages: sent 1011 (internal error) keepalive ping timeout; no close frame received
Exception in thread e2b-sandbox-refresh:
Traceback (most recent call last):
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connectionpool.py", line 793, in urlopen
    response = self._make_request(
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
  File "C:\Python39\lib\http\client.py", line 1349, in getresponse
    response.begin()
  File "C:\Python39\lib\http\client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "C:\Python39\lib\http\client.py", line 27

In [21]:
agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...
To solve this problem, we need to calculate the point of closest approach between the two aircraft based on their current positions, headings, and speeds. Given that both aircraft are flying at the same altitude and have no vertical speed (V/S), we can simplify the problem to a 2D plane. The aircraft are flying directly towards each other, one from the east and the other from the west, both at the same speed. We can calculate the time until the closest approach by finding the distance between them and dividing it by their closing speed (the sum of their ground speeds if they are moving directly towards each other).

First, we need to convert the positions from degrees, minutes, and seconds to decimal degrees to calculate the distance between them. Then, we can calculate the distance and use the ground speed to find the time until the closest approach. Since they are flying directly towards each other, the point of closest approach will be at the 

SyntaxError: invalid syntax (<unknown>, line 1)